## TODO - Needs fix

In [1]:
from tvb.interfaces.command.lab import *
from tvb.adapters.creators.allen_creator import AllenConnectomeBuilder
from tvb.adapters.creators.allen_creator import AllenConnectModel
from tvb.core.adapters.abcadapter import ABCAdapter
from tvb.core.services.operation_service import OperationService

2020-10-22 11:53:02,593 - WARNING - tvb.config.init.introspector_registry - Skipped Adapter(probably because MATLAB not found):<class 'tvb.adapters.analyzers.bct_adapters.DistanceDBIN'>
2020-10-22 11:53:02,595 - WARNING - tvb.config.init.introspector_registry - Skipped Adapter(probably because MATLAB not found):<class 'tvb.adapters.analyzers.bct_adapters.DistanceDWEI'>
2020-10-22 11:53:02,595 - WARNING - tvb.config.init.introspector_registry - Skipped Adapter(probably because MATLAB not found):<class 'tvb.adapters.analyzers.bct_adapters.DistanceNETW'>
2020-10-22 11:53:02,596 - WARNING - tvb.config.init.introspector_registry - Skipped Adapter(probably because MATLAB not found):<class 'tvb.adapters.analyzers.bct_adapters.DistanceRDA'>
2020-10-22 11:53:02,597 - WARNING - tvb.config.init.introspector_registry - Skipped Adapter(probably because MATLAB not found):<class 'tvb.adapters.analyzers.bct_adapters.DistanceRDM'>
2020-10-22 11:53:02,598 - WARNING - tvb.config.init.introspector_registr

For the next step to work correctly, you should have launched TVB Web GUI at least once, and fill the settings step.
At least one project needs to exist in your local environment for the following operations to exist in.

In [2]:
list_projects()

                    name    id
         Default_Project     1


In [4]:
## This ID of a project needs to exists in Db, and it can be taken from the WebInterface:
project = dao.get_project_by_id(1)
    
builder_instance = ABCAdapter.build_adapter_from_class(AllenConnectomeBuilder)

view_model = AllenConnectModel()
view_model.resolution = 100
view_model.weighting = 1
view_model.inj_f_thresh = 80
view_model.vol_thresh= 100000000

## launch an operation and have the results stored both in DB and on disk
operation_service = OperationService()
launched_operations = operation_service.fire_operation(builder_instance,
                                                  project.administrator,
                                                  project.id,
                                                  view_model = view_model)

2020-10-22 11:59:33,702 - INFO - tvb.core.services.operation_service - Starting operation AllenConnectomeBuilder
2020-10-22 11:59:33,753 - INFO - tvb.core.services.operation_service - Finished operation launch:AllenConnectomeBuilder


In [23]:
launched_operation = launched_operations[0]
# wait for the operation to finish
while not launched_operation.has_finished:
    sleep(5)
    launched_operation = dao.get_operation_by_id(launched_operation.id)

launched_operation

<Operation('4b7a893000d144ab883e54d9bf4a0e2c', e760979e-1444-11eb-8ae7-faffc245676e, 2,'1','64','2020-10-22 11:59:33.712976','2020-10-22 11:59:38.578661', '2020-10-22 13:39:48.233128','5-FINISHED',True, 'None', '2020-10-22,13-39-48', '', -1)>

At this point, your operation of Allen DB import should be running in the background.
If you launch TVB web GUI, and you go to Project -> Operations page, you should see the new operation running (state color BLUE).

When done, your results will be ready for inspection in TVB web GUI.
Do not close your computer until the operation has finished. If you do so, then relaunch after you restart. Downloaded files will not be lost, but found by the new operation.


In [24]:
from tvb.core.services.project_service import ProjectService
results = ProjectService().get_results_for_operation(op.id)
results

[<DataType(33, 17a5aca1646d49a6bff3b1da783e4461, ConnectivityIndex, tvb.adapters.datatypes.db.connectivity, John Doe, RAW_DATA, None, None, , , , )>,
 <DataType(34, c2b8eaa0c81043929874ec0cbf916f5c, VolumeIndex, tvb.adapters.datatypes.db.volume, John Doe, RAW_DATA, None, None, , , , )>,
 <DataType(35, 378beafb864b4e64aa9aa115fbb4dd57, RegionVolumeMappingIndex, tvb.adapters.datatypes.db.region_mapping, John Doe, RAW_DATA, None, None, , , , )>,
 <DataType(36, 4565179ffa1d4173b3f0acb500614b32, StructuralMRIIndex, tvb.adapters.datatypes.db.structural, John Doe, RAW_DATA, None, None, , , , )>]